#### Bobba 2022 abridged QC

**Objective**: Run QC steps without normalisation/UMAP for Bobba 2022 data

**Developed by**: Mairi McClean

**Affiliation**: Institute of Computational Biology, Helmholtz Zentrum Munich

**v230308**

In [1]:
import anndata
import logging
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sb
import scrublet as scr
import matplotlib.pyplot as plt
import pybiomart as biomart
from matplotlib import colors
from matplotlib import rcParams

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 160, color_map = 'RdPu', dpi_save = 180, vector_friendly = True, format = 'svg')

-----
anndata     0.8.0
scanpy      1.9.2
-----
PIL                 8.2.0
anyio               NA
appnope             0.1.3
arrow               1.2.3
asttokens           NA
attr                22.2.0
babel               2.12.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
certifi             2022.12.07
cffi                1.15.1
charset_normalizer  2.1.1
colorama            0.4.6
comm                0.1.2
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.6
decorator           5.1.1
executing           1.2.0
fastjsonschema      NA
fqdn                NA
future              0.18.3
h5py                3.8.0
hypergeom_ufunc     NA
idna                3.4
igraph              0.10.4
ipykernel           6.21.2
ipython_genutils    0.2.0
isoduration         NA
jedi                0.18.2
jinja2              3.1.2
joblib              1.2.0
json5               NA
jsonpointer         2.3
jsonschema    

### Read in datasets

##### Control data

In [7]:
adata_1 = sc.read_10x_mtx('/Volumes/LaCie/data_lake/Mairi_example/INBOX/sc_downloads/bobba_2022/GSE176423_RAW/ctl/')
adata_1

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


AnnData object with n_obs × n_vars = 661 × 27998
    var: 'gene_ids'

##### h445y

In [10]:
adata_2 = sc.read_10x_mtx('/Volumes/LaCie/data_lake/Mairi_example/INBOX/sc_downloads/bobba_2022/GSE176423_RAW/h445y/')
adata_2


--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


AnnData object with n_obs × n_vars = 1849 × 27998
    var: 'gene_ids'

##### hn878

In [11]:
adata_3 = sc.read_10x_mtx('/Volumes/Lacie/data_lake/Mairi_example/INBOX/sc_downloads/bobba_2022/GSE176423_RAW/hn878/')
adata_3


--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


AnnData object with n_obs × n_vars = 858 × 27998
    var: 'gene_ids'

In [12]:
adata_1.var


,gene_ids
Xkr4,ENSMUSG00000051951
Gm1992,ENSMUSG00000089699
Gm37381,ENSMUSG00000102343
Rp1,ENSMUSG00000025900
Rp1-1,ENSMUSG00000109048
...,...
AC168977.1,ENSMUSG00000079808
PISD,ENSMUSG00000095041
DHRSX,ENSMUSG00000063897
Vmn2r122,ENSMUSG00000096730


In [13]:
adata_2.var

,gene_ids
Xkr4,ENSMUSG00000051951
Gm1992,ENSMUSG00000089699
Gm37381,ENSMUSG00000102343
Rp1,ENSMUSG00000025900
Rp1-1,ENSMUSG00000109048
...,...
AC168977.1,ENSMUSG00000079808
PISD,ENSMUSG00000095041
DHRSX,ENSMUSG00000063897
Vmn2r122,ENSMUSG00000096730


In [14]:
adata_3.var

,gene_ids
Xkr4,ENSMUSG00000051951
Gm1992,ENSMUSG00000089699
Gm37381,ENSMUSG00000102343
Rp1,ENSMUSG00000025900
Rp1-1,ENSMUSG00000109048
...,...
AC168977.1,ENSMUSG00000079808
PISD,ENSMUSG00000095041
DHRSX,ENSMUSG00000063897
Vmn2r122,ENSMUSG00000096730


In [15]:
adata_1.obs

""
AAACCCATCTGAGGCC-1
AAACGAAAGAGGTTTA-1
AAAGAACAGTTGTCAC-1
AAAGAACCAAGTGGCA-1
AAAGAACGTAGCGTCC-1
...
TTTGACTCAATAGTAG-1
TTTGACTCACAAAGCG-1
TTTGACTTCTCCCAAC-1
TTTGTTGCACACAGAG-1


In [16]:
adata_2.obs

""
AAACGAAAGCGATTCT-1
AAACGAAAGTCGAAGC-1
AAACGAACAAAGCTAA-1
AAACGAAGTATCGTTG-1
AAACGCTAGCACGTCC-1
...
TTTGATCTCTACCACC-1
TTTGGAGAGACCATGG-1
TTTGGAGGTACCCGCA-1
TTTGGAGTCTCATAGG-1


In [17]:
adata_3.obs

""
AAACGAAGTCGCTTGG-1
AAACGAATCCATCACC-1
AAAGAACTCATTTGCT-1
AAAGGGCCAACAAGAT-1
AAAGGGCCAGTAACGG-1
...
TTTCAGTTCGAGGCAA-1
TTTCATGAGGTTAGTA-1
TTTCCTCTCTCCACTG-1
TTTGATCTCCCTGGTT-1


#### Concatenate objects?

In [19]:
adata_total = adata_1.concatenate(adata_2, adata_3, join="inner", batch_key="batch")
adata_total

/Users/mairi.mcclean/mambaforge/envs/scanpy_local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/Users/mairi.mcclean/mambaforge/envs/scanpy_local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/Users/mairi.mcclean/mambaforge/envs/scanpy_local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being co

AnnData object with n_obs × n_vars = 3368 × 27998
    obs: 'batch'
    var: 'gene_ids'

### Check concatenated anndata object


In [ ]:
# checks the number of molecules counted per barcode(?), using "donor" as proxy(?)
adata_1.obs['donor'].value_counts()

In [ ]:
# check omics mode
adata_1.obs['data_type'].value_counts

In [ ]:
# check tissue type


In [ ]:
# Make names unique to avoid computational issues re repeated gene names
adata.var_names_make_unique()
sample_object = adata.copy()
sample_object

### Replace gene symbols
#### Performed to ensure that index concatenation is smoother

> required for this particular dataset?

In [ ]:
sample_object.var['gene_id'] = sample_object.var.index.copy()
sample_object.var.set_index('gene_name', inplace = True)
sample_object.var.head()

In [ ]:
sample_object.var_names = [str(i) for i in sample_object.var_names]
sample_object.var_names_make_unique()

### Top 20 scatterplot

In [ ]:
sc.pl.highest_expr_genes(sample_object, n_top=20)

### Filter cells by gene number

In [ ]:
sc.pp.filter_cells(sample_object, min_genes = 200)
print(sample_object.n_obs, sample_object.n_vars)

In [ ]:
sample_object.shape

In [ ]:
sample_object.var

### Filter genes by cell number

In [ ]:
# sc.pp.filter_genes(sample_object, min_cells = 3)
# sample_object.shape

# removed because of erroneous QC visualisation graphs

### QC metrics

In [ ]:
sample_object.var['mt'] = sample_object.var_names.str.startswith('MT-')
sample_object.var['ribo'] = sample_object.var_names.str.startswith(("RPS","RPL"))
sample_object.var

In [ ]:
sc.pp.calculate_qc_metrics(sample_object, qc_vars = ['mt', 'ribo'], percent_top = None, log1p = False, inplace = True)

In [ ]:
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
# add the total counts per cell as observations-annotation to adata

mito_genes = sample_object.var_names.str.startswith('MT-')
sample_object.obs['percent_mt2'] = np.sum(
    sample_object[:, mito_genes].X, axis = 1).A1 / np.sum(sample_object.X, axis = 1).A1
sample_object.obs['n_counts'] = sample_object.X.sum(axis = 1).A1

In [ ]:
sample_object

### QC visualisation

In [ ]:
sample_object.var_names

In [ ]:
sc.pl.violin(sample_object, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(sample_object, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo'],
             jitter=0.4, groupby = 'status', rotation = 45)

In [ ]:
sc.pl.scatter(sample_object, x='total_counts', y='n_genes_by_counts', color = "donor")

### Add sample sex covariate

In [ ]:
annot = sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "external_gene_name", "start_position", "end_position", "chromosome_name"],
    ).set_index("external_gene_name")

In [ ]:
annot.head()

In [ ]:
chrY_genes = sample_object.var_names.intersection(annot.index[annot.chromosome_name == "Y"])
chrY_genes

In [ ]:
sample_object.obs['percent_chrY'] = np.sum(
    sample_object[:, chrY_genes].X, axis = 1).A1 / np.sum(sample_object.X, axis = 1).A1 * 100

sample_object.obs["XIST-counts"] = sample_object.X[:,sample_object.var_names.str.match('XIST')].toarray()


### Cell cycle scores

In [ ]:
!if [ ! -f /Users/mairi.mcclean/data/qc_files/cell_cycle_gene.txt ]; then curl -o /Users/mairi.mcclean/data/qc_files/cell_cycle_gene.txt https://raw.githubusercontent.

In [ ]:
cell_cycle_genes = [x.strip() for x in open('/Users/mairi.mcclean/data/qc_files/cell_cycle_gene')]
print(len(cell_cycle_genes))

# Split into 2 lists
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

cell_cycle_genes = [x for x in cell_cycle_genes if x in sample_object.var_names]
print(len(cell_cycle_genes))

In [ ]:
# create other anndata object for score calculation

adata_log = anndata.AnnData(X = sample_object.X,  var = sample_object.var, obs = sample_object.obs)
sc.pp.normalize_total(adata_log, target_sum = 1e6, exclude_highly_expressed = True)
sc.pp.log1p(adata_log)

In [ ]:
sc.tl.score_genes_cell_cycle(adata_log, s_genes = s_genes, g2m_genes = g2m_genes)
sc.pl.violin(adata_log, ['S_score', 'G2M_score'],
             jitter = 0.4, groupby = 'donor', rotation = 45)

In [ ]:
sample_object.obs['S_score'] = adata_log.obs['S_score']
sample_object.obs['G2M_score'] = adata_log.obs['G2M_score']
sample_object

### Predict doublets

In [ ]:
holder = np.zeros((sample_object.shape[0],))
for smp in np.unique(sample_object.obs['sample']):
    if smp == []:
        continue
    adata_smp = sample_object[sample_object.obs['sample'] == smp]
    scrub = scr.Scrublet(adata_smp.X)
    adata_smp.obs['doublet_scores'], adata_smp.obs['predicted_doublets'] = scrub.scrub_doublets()
    holder[sample_object.obs['sample'] == smp] = adata_smp.obs['predicted_doublets']
sample_object.obs['predicted_doublets'] = holder

sum(sample_object.obs['predicted_doublets'])
sample_object

In [ ]:
scrub.plot_histogram()

In [ ]:
#check if our predicted doublets also have more detected genes in general

sc.pl.violin(sample_object, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'predicted_doublets'],
             jitter = 0.2, groupby = 'donor', rotation = 45)

### Prepare counts for individual slots

In [ ]:
sample_object.raw = sample_object.copy()
sample_object.layers['counts'] = sample_object.X.copy()
sample_object.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(sample_object, inplace = False)["X"]
)
sample_object

### Export object

In [ ]:
sample_object.write('/Volumes/Lacie/data_lake/Mairi_example/processed_files/abridged_qc/mice/Bobba2022_mm230308.h5ad')